The goal of this notebook is to use SHAP to assess feature importance in more complex models

In [54]:
import shap

In [60]:
import pickle
import shap
import pandas as pd
from predict_protein import download_cptac, select_features, train_model, utils
import matplotlib.pyplot as plt
import numpy as np

model_loc = '/Volumes/Ginkgo_Data/Data/2021-11_CPTACTrainedModels/out3'

In [5]:
cptac_list = download_cptac.download_cptac(n_tumors=8)
tumor_df = pd.concat(cptac_list)
tm = train_model.LearnCPTAC(tumor_df)


cptac warning: transcriptomics data was not found for the following samples, so transcriptomics data columns were filled with NaN for these samples: C3L-00563.N, C3L-00605.N, C3L-00769.N, C3L-00770.N, C3L-00771.N, C3L-00930.N, C3L-00947.N, C3L-00963.N, C3L-01246.N, C3L-01249.N, C3L-01252.N, C3L-01256.N, C3L-01257.N, C3L-01744.N, C3N-00200.N, C3N-00729.N, C3N-01211.N, NX1.N, NX10.N, NX11.N, NX12.N, NX13.N, NX14.N, NX15.N, NX16.N, NX17.N, NX18.N, NX2.N, NX3.N, NX4.N, NX5.N, NX6.N, NX7.N, NX8.N, NX9.N (/Users/edwardlau/PythonProjects/CPTAC_Protein/predict_protein/download_cptac.py, line 38)
cptac warning: proteomics data was not found for the following samples, so proteomics data columns were filled with NaN for these samples: 01OV002, 01OV008, 01OV010, 01OV013, 01OV019, 01OV045, 01OV046, 01OV049, 02OV035, 02OV040, 02OV042, 02OV045, 04OV041, 11OV009, 13OV004, 17OV019, 17OV034, 22OV001, 26OV010 (/Users/edwardlau/PythonProjects/CPTAC_Protein/predict_protein/download_cptac.py, line 42)
cptac

In [6]:
def get_model(tm_obj, model_loc, model, protein_todo):

    with open(model_loc, 'rb') as f:
        model_results = pickle.load(f)

    protein_model  = [result['model'] for result in model_results if result['metrics'].index[0] == protein_todo][0]

    # Get features and then do train test split again
    tm_obj.included_features = model
    print(tm_obj.included_features)
    x_train, x_test, y_train, y_test = tm_obj.get_train_test(protein_to_do=protein_todo)

    # Output a pandas data frame with y (actual mass spec protein level) and y_hat (predicted from transcript(s))
    train_df = pd.DataFrame({'y': y_train, 'yhat': protein_model.predict(x_train[protein_model.feature_names_in_]), 'set': 'train'})
    test_df = pd.DataFrame({'y': y_test, 'yhat': protein_model.predict(x_test[protein_model.feature_names_in_]), 'set': 'test'})

    return protein_model, x_test # pd.concat([train_df, test_df])


In [76]:
for model in ['string', ]: # 'single', 'corum', 'stringhi', 'string'
    for protein in ['CMC1', 'PCCB', 'PSMG2', 'SMCR8']:
        m, X = get_model(tm_obj=tm,
                             model_loc=f'{model_loc}/tumor8_boosting_{model}.p',
                             model=model,
                             protein_todo=protein,
                        )
        # Reorder the X data frame
        X = X[m.feature_names_in_]
        #%% SHAP
        X100 = shap.utils.sample(X, 100)
        explainer = shap.Explainer(m.predict, X100)
        shap_values = explainer(X)

        #%%# Plot SHAP
        shap.plots.beeswarm(shap_values, max_display=20, show=False)
        plt.tight_layout()
        plt.savefig(f'08_SHAPFeatureImportance_Output/{protein}.pdf')
        plt.close()
        shap.summary_plot(shap_values, X, plot_type="bar", show=False)
        plt.tight_layout()
        plt.tight_layout()
        plt.savefig(f'08_SHAPFeatureImportance_Output/{protein}_bar.pdf')
        plt.close()

string


Permutation explainer: 257it [01:55,  2.04it/s]                         


string


Exception: max_evals=500 is too low for the Permutation explainer, it must be at least 2 * num_features + 1 = 803!

In [68]:
shap.force_plot(explainer, shap_values, X)

AssertionError: visualize() can only display Explanation objects (or arrays of them)!